In [1]:
%scala
dbutils.fs.mount(
  source = "wasbs://<Container name>@<Storage account name>.blob.core.windows.net",
  mountPoint = "/mnt/data",
  extraConfigs = Map("<conf-key>" -> "<Storage account key>"))

at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.createAzureStorageSession(AzureNativeFileSystemStore.java:1031)
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.initialize(AzureNativeFileSystemStore.java:482)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.initialize(NativeAzureFileSystem.java:1292)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.verifyAzureFileSystem(DBUtilsCore.scala:497)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:446)
	at com.databricks.dbutils_v1.impl.DbfsUtilsImpl.mount(DbfsUtilsImpl.scala:85)
	at line1d15cc6fe3754116918060b0f68e0e8a52.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-1410223159474767:1)
	at line1d15cc6fe3754116918060b0f68e0e8a52.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-1410223159474767:47)
	at line1d15cc6fe3754116918060b0f68e0e8a52.$read$$iw$$iw$$iw$$iw.<init>(command-1410223159474767:49)
	at line1d15cc6fe3754116918060b0f68e0e8a52.$read$$iw$$iw$$iw.<init>(command-1410223159474767:51)
	at line1d15cc6fe3754116918060b0f68e0e8a52.$read$$iw$$iw.<init>(command-1410223159474767:53)
	at line1d15cc6fe3754116918060b0f68e0e8a52.$read$$iw.<init>(command-1410223159474767:55)
	at line1d15cc6fe3754116918060b0f68e0e8a52.$read.<init>(command-1410223159474767:57)
	at line1d15cc6fe3754116918060b0f68e0e8a52.$read$.<init>(command-1410223159474767:61)
	at line1d15cc6fe3754116918060b0f68e0e8a52.$read$.<clinit>(command-1410223159474767)
	at line1d15cc6fe3754116918060b0f68e0e8a52.$eval$.$print$lzycompute(<notebook>:7)
	at line1d15cc6fe3754116918060b0f68e0e8a52.$eval$.$print(<notebook>:6)
	at line1d15cc6fe3754116918060b0f68e0e8a52.$eval.$print(<notebook>)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:793)
	at scala.tools.nsc.interpreter.IMain$Request.loadAndRun(IMain.scala:1054)
	at scala.tools.nsc.interpreter.IMain$WrappedRequest$$anonfun$loadAndRunReq$1.apply(IMain.scala:645)
	at scala.tools.nsc.interpreter.IMain$WrappedRequest$$anonfun$loadAndRunReq$1.apply(IMain.scala:644)
	at scala.reflect.internal.util.ScalaClassLoader$class.asContext(ScalaClassLoader.scala:31)
	at scala.reflect.internal.util.AbstractFileClassLoader.asContext(AbstractFileClassLoader.scala:19)
	at scala.tools.nsc.interpreter.IMain$WrappedRequest.loadAndRunReq(IMain.scala:644)
	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:576)
	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:572)
	at com.databricks.backend.daemon.driver.DriverILoop.execute(DriverILoop.scala:215)
	at com.databricks.backend.daemon.driver.ScalaDriverLocal$$anonfun$repl$1.apply$mcV$sp(ScalaDriverLocal.scala:197)
	at com.databricks.backend.daemon.driver.ScalaDriverLocal$$anonfun$repl$1.apply(ScalaDriverLocal.scala:197)
	at com.databricks.backend.daemon.driver.ScalaDriverLocal$$anonfun$repl$1.apply(ScalaDriverLocal.scala:197)
	at com.databricks.backend.daemon.driver.DriverLocal$TrapExitInternal$.trapExit(DriverLocal.scala:694)
	at com.databricks.backend.daemon.driver.DriverLocal$TrapExit$.apply(DriverLocal.scala:647)
	at com.databricks.backend.daemon.driver.ScalaDriverLocal.repl(ScalaDriverLocal.scala:197)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$9.apply(DriverLocal.scala:381)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$9.apply(DriverLocal.scala:358)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:235)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:230)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(Dr

In [2]:
%scala

val df = spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .option("delimiter", ",")
  .csv("/mnt/data/housing.csv")

df: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 8 more fields]

In [3]:
%scala
import org.apache.spark.eventhubs._

// Build connection string with the above information
val connectionString = ConnectionStringBuilder("<Event hub connection string>")
  .setEventHubName("<Event hub name>")
  .build

val customEventhubParameters =
  EventHubsConf(connectionString)
  .setMaxEventsPerTrigger(5)

val streaming_df = spark.readStream.format("eventhubs").options(customEventhubParameters.toMap).load()

streaming_df.join(df)

import org.apache.spark.eventhubs._
import org.apache.spark.sql.streaming.Trigger
connectionString: String = Endpoint=sb://mlnetpredict.servicebus.windows.net/;EntityPath=streamingdemo;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=qRsNbPwNLiNMxjBQY9wIDjKIzmRFkw1LhTEqw0TkC14=
customEventhubParameters: org.apache.spark.eventhubs.EventHubsConf = org.apache.spark.eventhubs.EventHubsConf@4704afa3
streaming_df: org.apache.spark.sql.DataFrame = [body: binary, partition: string ... 7 more fields]
res7: org.apache.spark.sql.DataFrame = [body: binary, partition: string ... 17 more fields]